# Time series forecasting

## Dataset

此份筆記本會使用由 <a href="https://www.bgc-jena.mpg.de" class="external">Max Planck Institute for Biogeochemistry</a> 所記錄的<a href="https://www.bgc-jena.mpg.de/wetter/" class="external">天氣資料集</a>來實作時間序列預測模型。

此份資料集包含 14 種不同的特徵，例如氣溫、大氣壓力和濕度。每 10 分鐘收集一次數據，全部資料含蓋由 2009 年至 2016 年之間收集的數據。

下面我們將實作幾組模型進行**每小時**的天氣預測。

Notebook 分成以下三個部分:
- EDA & Feature Engineering
- Data Windowing
- Models
    - Single-step Models: 預測**單個時間點**的數值
    - Multi-output Models: 預測**多個特徵**數值
    - Multi-step Models: 預測**多個時間點**的數值

## Importing Packages

In [ ]:
import os

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [ ]:
!wget -q 'https://github.com/TA-aiacademy/course_3.0/releases/download/TSRNN/jena_climate_2009_2016.csv.zip'
!unzip -q 'jena_climate_2009_2016.csv.zip'

資料集特徵除了 **Date Time** 之外還包含另外 14 種特徵

其說明可參照 <a href="https://www.bgc-jena.mpg.de/wetter/Weatherstation.pdf" class="external">Max Planck Institute for Biogeochemistry</a> 所提供之說明

<img src="https://i.imgur.com/hDrlewM.png" width="600">

In [ ]:
df = pd.read_csv('jena_climate_2009_2016.csv')

df.info()

In [ ]:
df.head(10)

In [ ]:
# 由 Date Time 欄位可得知貢料為每 10 分鐘收集一筆
# 由於我們要進行預測的時間間隔是以「每小時」為單位
# 所以將資料以小時為單位做切片俿理
# [start:stop:step] 從第 5 篳資料開始，每經過 6 個時間記錄點取一次資料

df = df[5::6]
df.head()

In [ ]:
type(df['Date Time'][5])

In [ ]:
# 將 Date Time 欄位獨立取出，並將其中資料由 str 轉換為 datetime

date_time = pd.to_datetime(df.pop('Date Time'), format='%d.%m.%Y %H:%M:%S')

## EDA & Feature Engineering

In [ ]:
# 針對 'T (degC)', 'p (mbar)', 'rho (g/m**3)' 等特徵畫出它們的時間變化圖

# 2009~2016
plot_cols = ['T (degC)', 'p (mbar)', 'rho (g/m**3)']
plot_features = df[plot_cols]
plot_features.index = date_time
_ = plot_features.plot(subplots=True)

In [ ]:
# 前 480 個小時(20 天)
plot_features = df[plot_cols][:480]
plot_features.index = date_time[:480]
_ = plot_features.plot(subplots=True)

In [ ]:
# 查看每個特徵的敘述統計量

df.describe().transpose()

從以上的統計量描述可以發現 (`wv (m/s)`) 和 (`max. wv (m/s)`) 的 `min` 值為 `-9999` ，這樣的數值很有可能是錯的

另外透過 `wd(deg)` 風向這個欄位可判斷風速應該要大於或等於零 (`>=0`)，因此我們將 (`wv (m/s)`) 和 (`max. wv (m/s)`) 中的 `-9999` 代換為零:

In [ ]:
# 將 'wv (m/s)' 中值等於 -9999.0 的值代換為 0.0
wv = df['wv (m/s)']
bad_wv = wv == -9999.0
wv[bad_wv] = 0.0

# 將 'max. wv (m/s)' 中值等於 -9999.0 的值代換為 0.0
max_wv = df['max. wv (m/s)']
bad_max_wv = max_wv == -9999.0
max_wv[bad_max_wv] = 0.0

# 最後可再檢查代換後兩個特徵的最三值是否 >= 0.0
df['wv (m/s)'].min(), df['max. wv (m/s)'].min()

### Wind

In [ ]:
# 畫出 'wd (deg)' 與 'wv (m/s)' 的 2D 直方圖

plt.hist2d(df['wd (deg)'], df['wv (m/s)'], bins=(50, 50), vmax=400)
plt.colorbar()
plt.xlabel('Wind Direction [deg]')
plt.ylabel('Wind Velocity [m/s]')

風向 `wd (deg)` 採 0°~360° 為記錄資料。這樣的記錄方式對模型來說不是一件好事，因為 360° 與  0° 彼此應該是靠近且平滑連接的。

我們可以利用「風向」和「風速」轉換出風向量的「x分量」與「y分量」

<img src="https://i.imgur.com/w3bZcUP.png" width="300">

In [ ]:
wv = df.pop('wv (m/s)')
max_wv = df.pop('max. wv (m/s)')

# 將角度由度度量轉換為弧度度量
wd_rad = df.pop('wd (deg)')*np.pi / 180

# 計算風的「x分量」與「y分量」 
df['Wx'] = wv*np.cos(wd_rad)
df['Wy'] = wv*np.sin(wd_rad)

# 計算最大風的「x分量」與「y分量」
df['max Wx'] = max_wv*np.cos(wd_rad)
df['max Wy'] = max_wv*np.sin(wd_rad)

In [ ]:
# 畫出 'Wx', 'Wy' 的 2D 直方圖

plt.hist2d(df['Wx'], df['Wy'], bins=(50, 50), vmax=400)
plt.colorbar()
plt.xlabel('Wind X [m/s]')
plt.ylabel('Wind Y [m/s]')
ax = plt.gca()
ax.axis('tight')

### Time

天氣類型的數據可能會與一天當中的時間或是一年當中的時間週期有相關性。

我們可以通過使用 sine / cosine transform 將「一天的時間」資訊和「一年的時間」資訊編碼起來形成時間資料。

In [ ]:
# 將 data_time 中的「日期時間」資料轉換成「秒(float)」(ex: 2009-01-01 01:00:00 ---> 1230771600.0)
timestamp_s = date_time.map(pd.Timestamp.timestamp)

# 分別計算一天與一年之秒數
day = 24*60*60
year = (365.2425)*day

# 分別進行一天與一年為週期的 sine / cosine 轉換
df['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
df['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
df['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
df['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))

In [ ]:
# 畫出一天 (24小時之內) 的時間變化
plt.plot(np.array(df['Day sin'])[:25])
plt.plot(np.array(df['Day cos'])[:25])
plt.xlabel('Time [h]')
plt.title('Time of day signal')

### Split the data

將資料以 `(70%, 20%, 10%)` 的比例切分成「訓練」、「驗證」與「測試」集。

要特別注意，在切分前勿將資料打亂，以避免後續在取 window 時喪失時間相關性。

In [ ]:
n = len(df)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

num_features = df.shape[1]

### Data Normalization

In [ ]:
# 計算訓練集各特徵的平均數與標準差
train_mean = train_df.mean()
train_std = train_df.std()

# 分別對訓練、驗證、測試資料做 normalization
train_df = (train_df - train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

In [ ]:
# 進行完 normalization 後，可觀察各個特徵的數值分佈

df_std = (df - train_mean) / train_std
df_std = df_std.melt(var_name='Column', value_name='Normalized') # 將 df_std DataFrame 從 wide format 轉成 long format 

plt.figure(figsize=(16, 8))
ax = sns.violinplot(x='Column', y='Normalized', data=df_std)
_ = ax.set_xticklabels(df.keys(), rotation=90)

## Data windowing

對 DataFrame 中的資料依照時序進行窗口採樣 (Windowing)

對於 Windowing 而言，以下三個特性是應當考慮到的:

- input windows 和 lable windows 的時間長度（number of time steps）
- input windows 和 lable windows 的時間差 (time offset)
- 哪些 feature 被用作輸入(input)、標籤(label)或兩者都是 

我們可依據任務與使用的模型產生不同的 windows，以下是幾個例子:

1. 如果我們要利用前 24 小時的歷史資料來預測未來 24 小時後的天氣狀態，我們取 window 的方式應該如下:

<img src="https://i.imgur.com/LQmCJWk.png">

2. 如果我們要利用前 6 小時的歷史資料來預測未來 1 小時的天氣狀態，我們取 window 的方式應該如下:

<img src="https://i.imgur.com/zseRyiv.png">

接下來我們定義了一個 `WindowGenerator` 類。這個類能夠：

1. 處理上方圖表中所示的索引(`indexes`)和時間偏移量(`offsets`)。
2. 將 `windows` 分成 `(inputs, labels)` 對。
3. 繪製 `windows` 的內容。
4. 使用 `tf.data.Dataset` 從訓練、評估和測試數據中高效地生成這些窗口的批次。

### 1. Indexes and offsets

`WindowGenerator` 類中的 `__init__` 方法包含了 `input` 和 `label` 的**索引**所需的所有邏輯。

`WindowGenerator` 還接受訓練、驗證和測試集的 `DataFrame` 作為輸入。這些稍後將被轉換為 `windows` 的 `tf.data.Dataset`

In [ ]:
class WindowGenerator():
    def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
               label_columns=None):
        # 存入原始數據
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        # 將要預測的 label column 做索引編號
        # 如果 label_columns == None 則將 train_df 中的所有 columns 作索引編號
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in
                                        enumerate(label_columns)}
        self.column_indices = {name: i for i, name in
                               enumerate(train_df.columns)}

        # 進行 windows 的參數設定
        self.input_width = input_width                                                 # 設定 input window 長度
        self.label_width = label_width                                                 # 說定 label window 長度
        self.shift = shift                                                             # 設定 time offset 長度

        self.total_window_size = input_width + shift

        # slice(start, end, step): 返回一個 slice 物件，用於指定如何對一個 sequence 做切片
        self.input_slice = slice(0, input_width)                                       # 切分 input window 索引位置的切片對象
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width                   # label window 的起始位置
        self.labels_slice = slice(self.label_start, None)                              # 切分 label window 索引位置的切片對象
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    # 定義 WindowGenerator() 物件的表現形式，如此可以方便知道一個 WindowGenerator() 物件的相關資訊
    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])

下面我們創建兩 `WindowGenerator` 物件，分別對應到上面兩張 windows 的圖

In [ ]:
w1 = WindowGenerator(input_width=24, label_width=1, shift=24,
                     label_columns=['T (degC)'])
w1

In [ ]:
w2 = WindowGenerator(input_width=6, label_width=1, shift=1,
                     label_columns=['T (degC)'])
w2

### 2. Split

將 `windows` 分成 `(inputs, labels)` 對。

<img src="https://i.imgur.com/ou4R3SI.png">

In [ ]:
def split_window(self, windows):
    # 定義將 windows 分解成 input_windows 與 label_windows 的方法
    # windows 的 shape 分別表示 (batch, time, features)
    
    input_windows = windows[:, self.input_slice, :]
    label_windows = windows[:, self.labels_slice, :]
    
    # 挑選出有指定做為 label 的特徵
    if self.label_columns is not None:
        label_windows = tf.stack(
            [label_windows[:, :, self.column_indices[name]] for name in self.label_columns],
            axis=-1)

    # 做完切片後檢查一次 input_windows 和 label_windows 的 time shape 是否與設定的 input_width 與 label_width 一致
    input_windows.set_shape([None, self.input_width, None])
    label_windows.set_shape([None, self.label_width, None])

    return input_windows, label_windows

# 將 split_window 賦值給 WindowGenerator.split_window
WindowGenerator.split_window = split_window

In [ ]:
# 將 3 個切片疊在一起，其時間長度為 total_window_size
example_window = tf.stack([np.array(train_df[:w2.total_window_size]),
                           np.array(train_df[100:100+w2.total_window_size]),
                           np.array(train_df[200:200+w2.total_window_size])])

example_inputs, example_labels = w2.split_window(example_window)

print('All shapes are: (batch, time, features)')
print(f'Window shape: {example_window.shape}')
print(f'Inputs shape: {example_inputs.shape}')
print(f'Labels shape: {example_labels.shape}')

### 3. Plot

針對 `input_windows`, `label_windows`, `模型預測` 視覺化內容

In [ ]:
# 先將上面 example_inputs 和 example_labels 賦值給 w2.example
w2.example = example_inputs, example_labels

In [ ]:
def plot(self, model=None, plot_col='T (degC)', max_subplots=3):
    inputs, labels = self.example           # 給定 batch size 個時間區段的 inputs 和 labels
    plt.figure(figsize=(12, 8))
    plot_col_index = self.column_indices[plot_col]
    max_n = min(max_subplots, len(inputs))  # 設定 subplots 的數量
    for n in range(max_n):
        plt.subplot(max_n, 1, n+1)
        plt.ylabel(f'{plot_col} [normed]')
        
        # 畫出 Plot_col 的 input 資料
        # zorder 控制繪圖的順序，數值愈大愈晚畫上表示會在圖的愈上層
        plt.plot(self.input_indices, inputs[n, :, plot_col_index],
                 label='Inputs', marker='.', zorder=-10)

        if self.label_columns:
            label_col_index = self.label_columns_indices.get(plot_col, None)
        else:
            label_col_index = plot_col_index

        if label_col_index is None:
            continue

        # 畫出 plot_col 的 label 資料
        plt.scatter(self.label_indices, labels[n, :, label_col_index], 
                    edgecolors='k', label='Labels', c='#2ca02c', s=64)
        if model is not None:
            predictions = model(inputs)
            
            # 畫出模型的預測結果
            plt.scatter(self.label_indices, predictions[n, :, label_col_index],marker='X', 
                        edgecolors='k', label='Predictions',c='#ff7f0e', s=64)

        if n == 0:
            plt.legend()

    plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [ ]:
w2.plot()

也可以選其它的 column 來畫資料點，不過 `w2` 在創建時僅設定了 label 為 `T (degC)` 所以圖中只會呈現 `input_windows` 的資料

In [ ]:
w2.plot(plot_col='p (mbar)')

### 4. Create `tf.data.Dataset`s

下面的 `make_dataset` 方法會接收時間序列 DataFrame 並將其轉換為 `tf.data.Dataset` of `(input_window, label_window)`

使用 `tf.keras.utils.timeseries_dataset_from_array` 會返回 `tf.data.Dataset` 物件，它的每一個輸出是一個一個的 `window`

In [ ]:
def make_dataset(self, data):
    data = np.array(data, dtype=np.float32)
    ds = tf.keras.utils.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

    ds = ds.map(self.split_window)

    return ds

WindowGenerator.make_dataset = make_dataset

`WindowGenerator` 對象保存了訓練、驗證和測試數據。 

使用之前定義的 `make_dataset` 方法添加屬性以將它們作為 `tf.data.Datasets` 訪問。此外，添加一個標準示例批次以便於訪問和繪圖。

In [ ]:
@property
def train(self):
    return self.make_dataset(self.train_df)

@property
def val(self):
    return self.make_dataset(self.val_df)

@property
def test(self):
    return self.make_dataset(self.test_df)

@property
def example(self):
    # 獲取並緩存一批用於繪圖的 “inputs 和 labels 示例
    
    # getattr() 函数用於返回一个對象的属性值，此處用以返回 `_example` 屬性
    result = getattr(self, '_example', None)
    if result is None:
        
        # 未找到示例批次，因此從 `.val` 數據集中獲取一個
        result = next(iter(self.val))
        
        # 並緩存以備下次使用
        self._example = result
    return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

現在，`WindowGenerator` 對象可以訪問 `tf.data.Dataset` 對象，因此我們可以輕鬆地迭代數據

`Dataset.element_spec` 屬性告訴您數據集元素的結構、數據類型和形狀

In [ ]:
# 每個元素都是一個（input，label）序對
w2.train.element_spec

我們也可以從 `Dataset` 中迭代出具體的批次

In [ ]:
for example_inputs, example_labels in w2.train.take(1):
    print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
    print(f'Labels shape (batch, time, features): {example_labels.shape}')

## Models

### Single-step Models

僅根據當前條件預測未來某一特定時間 (某一小時）的值

下面我們以預測**單個特徵** `T (degC)` 的值為例作為說明

若是要以 RNN 設計 single-step models , 則訓練方式可分為兩種 `以單個時間點為 Label` 和 `以多個時間點為 Label`:

#### 以單個時間點為 Label

其中一種是 RNN 僅返回最後一個時間步的輸出，讓模型有時間在進行單個預測之前預熱其內部狀態, 如下圖:

<img src="https://i.imgur.com/KBbPdgK.png">

要使用此種作法須將 Keras RNN layer 中的 `return_sequences` 參數設為 `False`

In [ ]:
window_1h = WindowGenerator(
    input_width=24, label_width=1, shift=1,
    label_columns=['T (degC)'])

window_1h

In [ ]:
lstm_model_1h = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=False, input_shape=(24, 19)),
    # Shape => [batch, dense_units]
    tf.keras.layers.Dense(units=1),
    # Shape => [batch, 1, features](使 model prediction shape 與 label shape 一致)
    tf.keras.layers.Reshape([1, -1])
])

lstm_model_1h.summary()

In [ ]:
MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

    model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

    history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
    return history

In [ ]:
print('Input shape:', window_1h.example[0].shape)
print('Output shape:', lstm_model_1h(window_1h.example[0]).shape)

In [ ]:
history = compile_and_fit(lstm_model_1h, window_1h)

IPython.display.clear_output()

# tf.keras.Model.evaluate => (loss value, metrics values)
print('Val MAE:', lstm_model_1h.evaluate(window_1h.val, verbose=0)[1])
print('Test MAE:', lstm_model_1h.evaluate(window_1h.test, verbose=0)[1])

In [ ]:
window_1h.plot(lstm_model_1h)

#### 以多個時間點為 Label

另外一種使用 RNN 設計 single-step models 的方式為讓 RNN 為每個輸入返回一個輸出，如下圖:

<img src="https://i.imgur.com/iJ5Q1a7.png">

要使用此種作法須將 Keras RNN layer 中的 `return_sequences` 參數設為 `True`

In [ ]:
window_24h = WindowGenerator(
    input_width=24, label_width=24, shift=1,
    label_columns=['T (degC)'])

window_24h

In [ ]:
lstm_model_24h = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True, input_shape=(24, 19)),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=1)
])

lstm_model_24h.summary()

In [ ]:
print('Input shape:', window_24h.example[0].shape)
print('Output shape:', lstm_model_24h(window_24h.example[0]).shape)

In [ ]:
history = compile_and_fit(lstm_model_24h, window_24h)

IPython.display.clear_output()

# tf.keras.Model.evaluate => (loss value, metrics values)
print('Val MAE:', lstm_model_24h.evaluate(window_24h.val, verbose=0)[1])
print('Test MAE:', lstm_model_24h.evaluate(window_24h.test, verbose=0)[1])

In [ ]:
window_24h.plot(lstm_model_24h)

### Multi-output models

目前為止，以上模型都預測了**單個時間步長**的**單個輸出特徵** `T (degC)`

所有這些模型都可以轉換為預測**多個特徵**，只需更改輸出層中的單元數並調整 training windows 讓其包含標籤中的所有特徵

In [ ]:
multi_output_window_24h = WindowGenerator(
    input_width=24, label_width=24, shift=1)

In [ ]:
multi_output_lstm_model_24h = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=num_features)
])

In [ ]:
print('Input shape:', multi_output_window_24h.example[0].shape)
print('Output shape:', multi_output_lstm_model_24h(multi_output_window_24h.example[0]).shape)

In [ ]:
history = compile_and_fit(multi_output_lstm_model_24h, multi_output_window_24h)

IPython.display.clear_output()
print('Val MAE:', multi_output_lstm_model_24h.evaluate( multi_output_window_24h.val, verbose=0)[1])
print('Test MAE:', multi_output_lstm_model_24h.evaluate( multi_output_window_24h.test, verbose=0)[1])

In [ ]:
multi_output_window_24h.plot(multi_output_lstm_model_24h, plot_col='p (mbar)')

#### Residual connections

我們也可以設計殘差結構，讓模型預測下個時間點的變化量而非預測下個時間點的數值，這也是時間序列中常見的作法

<img src="https://i.imgur.com/1fwwTBC.png">

In [ ]:
class ResidualWrapper(tf.keras.Model):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def call(self, inputs, *args, **kwargs):
        delta = self.model(inputs, *args, **kwargs)

        # 每個時間黸的預測是前一個時間點的輸入(inputs)加上模型計算的差值(delta)。
        return inputs + delta

In [ ]:
multi_output_residual_lstm_24h = ResidualWrapper(
    tf.keras.Sequential([
    tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.Dense(
        num_features,
        # delta 的值一開始應該要小
        # 因此設定 output layer 初始化權重為零
        kernel_initializer=tf.initializers.zeros())
]))

In [ ]:
history = compile_and_fit(multi_output_residual_lstm_24h, multi_output_window_24h)

IPython.display.clear_output()
print('Val MAE:', multi_output_residual_lstm_24h.evaluate(multi_output_window_24h.val, verbose=0)[1])
print('Test MAE:', multi_output_residual_lstm_24h.evaluate( multi_output_window_24h.test, verbose=0)[1])

In [ ]:
multi_output_window_24h.plot(multi_output_residual_lstm_24h, plot_col='p (mbar)')

### Multi-step models

前面做的模型不論做的是 single-output models 還是 multi-output models 所作的都是 **single-step models** <br>
也就是模型做的預測是**未來一個時間步長**的值 <br>
接下來將著眼於如何擴展這些模型以進行**多時間步長**預測

In [ ]:
OUT_STEPS = 24
multi_step_window_24h = WindowGenerator(input_width=24, 
                                        label_width=OUT_STEPS, 
                                        shift=OUT_STEPS)

multi_step_window_24h

In [ ]:
multi_step_window_24h.plot()

#### RNN

RNN 可以學習使用長期的歷史輸入。模型會累積 24 小時的 internal state，然後再對接下來的 24 小時進行單一預測 <br>
在這種單次格式中，LSTM 只需要在最後一個時間步產生一個輸出，所以在 `tf.keras.layers.LSTM` 中設置 `return_sequences=False`

<img src="https://i.imgur.com/vCFIkc2.png">

In [ ]:
multi_step_lstm_model_24h = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units].
    tf.keras.layers.LSTM(32, return_sequences=False),
    # Shape => [batch, out_steps*features].
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features].
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

In [ ]:
history = compile_and_fit(multi_step_lstm_model_24h, multi_step_window_24h)

IPython.display.clear_output()
print('Val MAE:', multi_step_lstm_model_24h.evaluate(multi_step_window_24h.val, verbose=0)[1])
print('Test MAE:', multi_step_lstm_model_24h.evaluate(multi_step_window_24h.test, verbose=0)[1])

In [ ]:
multi_step_window_24h.plot(multi_step_lstm_model_24h)

#### Autoregressive model
構建 Autoregressive RNN 模型，模型從 `OUT_STEPS` 開始進行後面每個時間點的預測，每一步模型會參考前一步的預測再生成下一步的預測<br>
由於自迴歸的機制在 `tf.keras.layers.LSTM` 並沒有支援，這樣的機制要從較低級別的 `tf.keras.layers.LSTMCell` 進行單時間步運作的設計與管理

`RNN Layer`可用於一次性的處理整批序列，與 `RNN Layer`不同的是`RNN Cell`僅會進行單個時間步長的訊息處理<br>
可以把`RNN Layer`想成是將`RNN Cell`以 `for` loop 包裝起來，例如: `RNN(LSTMCell(10))`，如此將 Cell 包裝起來後就可以像 Layer 一樣一次性地處理整批序列

<img src="https://i.imgur.com/vXXugBd.png">

In [ ]:
class FeedBack(tf.keras.Model):
    def __init__(self, units, out_steps):
        super().__init__()
        self.out_steps = out_steps
        self.units = units
        self.lstm_cell = tf.keras.layers.LSTMCell(units)
        # 將 LSTMCell 包裝 RNN Layer 中
        self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
        self.dense = tf.keras.layers.Dense(num_features)

In [ ]:
feedback_model = FeedBack(units=32, out_steps=OUT_STEPS)

In [ ]:
def warmup(self, inputs):
    # inputs.shape => (batch, time, features)
    # x.shape => (batch, lstm_units)
    x, *state = self.lstm_rnn(inputs)

    # predictions.shape => (batch, features)
    prediction = self.dense(x)
    return prediction, state

FeedBack.warmup = warmup

In [ ]:
prediction, state = feedback_model.warmup(multi_step_window_24h.example[0])
prediction.shape

In [ ]:
def call(self, inputs, training=None):
    # 用以儲存每個時間點的 output
    predictions = []
    # 將 inputs 給 warmup 作用回傳出 prediction 與 state
    prediction, state = self.warmup(inputs)

    # 將第一次的預測結果放入 predictions list 中
    predictions.append(prediction)

    # 執行後續的 autoregressive prediction 過程
    for n in range(1, self.out_steps):
        # 使用上一個 prediction 作為 input
        x = prediction
        # 執行一次 lstm 
        x, state = self.lstm_cell(x, states=state,
                                  training=training)
        # 將 lstm 的 output 過一次 dense layer 進行預測
        prediction = self.dense(x)
        # Add the prediction to the output.
        predictions.append(prediction)

    # predictions.shape => (time, batch, features)
    predictions = tf.stack(predictions)
    # predictions.shape => (batch, time, features)
    predictions = tf.transpose(predictions, [1, 0, 2])
    return predictions

FeedBack.call = call

In [ ]:
print('Output shape (batch, time, features): ', feedback_model(multi_step_window_24h.example[0]).shape)

In [ ]:
history = compile_and_fit(feedback_model, multi_step_window_24h)

IPython.display.clear_output()
print('Val MAE:', feedback_model.evaluate(multi_step_window_24h.val, verbose=0)[1])
print('Test MAE:', feedback_model.evaluate(multi_step_window_24h.test, verbose=0)[1])

In [ ]:
multi_step_window_24h.plot(feedback_model)

## References

<a href="https://www.tensorflow.org/tutorials/structured_data/time_series#setup" class="external">TensorFlow 官網 Time Series Forcasting</a>.